## Processo para utlização da biblioteca Selenium para webscraping no site Quinto Andar

# Nessa parte vamos importar bibliotecas necessárias para o codigo

Pode se perceber que não utlizamos todas nesse script, porem foi deixado caso querem usar outras estrategias para o script

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import re
import time
import pandas as pd
import numpy as np
import requests
from time import sleep
from tqdm import tqdm

# Buscando os dados na web

AQUI IREMOS SALVAR TODOS OS NOMES DE CLASSES HTML PARA BUSCA  
Cada classe foi buscada no site quinto andar, fazendo a analise html do site

Preço do apartamento = CozyTypography xih2fc EKXjIf EqjlRj  
IPTU + Cond = CozyTypography xih2fc _72Hu5c Ci-jp3  
caracteristicas = CozyTypography o6Ojuq xih2fc EKXjIf A68t3o  
tipo  = Cozy__CardTitle-Metadata Dg2zLY  
endereço = CozyTypography xih2fc _72Hu5c Ci-jp3  




Vamos configurar o nosso browser para a abertura do site

In [2]:
driver = webdriver.Chrome()
driver.get('https://www.quintoandar.com.br/comprar/imovel/belo-horizonte-mg-brasil')

Iterando sobre o Botão ver mais para carregar todo elemento html do site.  

Lembrando que caso esse passo não seja feito vamos pegar apenas os empreendimentos carregados no html que ja esta carregado de inicio  

Caso seja utlizada a Biblioteca BeautifulSoup é necessario interagir com o botão ver mais utlizando a selenium, alem de passar a pagina para  
html e utilizar o metodo find_all ou selec ao inves de find_element  

A parte do range deve ser alterada para conseguir pegar todos os elementos, precisa ser trocado de acordo com o volume de apartamentos na região.  

No sleep deve ser alterado de forma que de tempo do seu computador carrega toda a pagina para executar o clique no botão novamente  

Aqui também utilizamos a biblioteca tqdm para saber qaunto do processo ja foi executado

In [ ]:
for i in tqdm(range(500), desc="Processando", unit=" clique"):
    botao = driver.find_element(By.CLASS_NAME, 'MuiBox-root.mui-d8ybqx')
    botao.click()
    sleep(5)

Usando as classes escolhidas no html, lembrando que os espaços precisam ser substuitos por pontos no Selenium.  

Caso use BeautifulSoup não é necessario a substituição

In [4]:
emp_valor = driver.find_elements(By.CLASS_NAME, 'CozyTypography.xih2fc.EKXjIf.EqjlRj')
emp_iptu_cond = driver.find_elements(By.CLASS_NAME, 'Cozy__CardTitle-Subtitle.JyjznE')
emp_caracter = driver.find_elements(By.CLASS_NAME, 'CozyTypography.o6Ojuq.xih2fc.EKXjIf.A68t3o')
emp_tipo = driver.find_elements(By.CLASS_NAME, 'Cozy__CardTitle-Metadata.Dg2zLY')
emp_end = driver.find_elements(By.CSS_SELECTOR, 'h2.CozyTypography.xih2fc._72Hu5c.Ci-jp3')

Agora criei uma lista para cada variavel onde irei armazenar os valores das colunas para aplicar a um DataFrame

In [6]:
valores = [valor.text for valor in emp_valor]
iptu_condicoes = [iptu.text for iptu in emp_iptu_cond]
caracteristicas = [caracter.text for caracter in emp_caracter]
tipos = [tipo.text for tipo in emp_tipo]
enderecos = [end.text for end in emp_end]

Aqui escolhi criar um Dicionario para para amazenar os valores e os nomes das colunas do Dataframe

In [7]:
data_dict = {
    'Valor': valores,
    'IPTU_Condicao': iptu_condicoes,
    'Caracteristicas': caracteristicas,
    'Tipo': tipos,
    'Endereco': enderecos
}

In [90]:
df = pd.DataFrame(data_dict)

# Tratamento dos dados

Nessa primeira parte vamos tratar os dados e deixa-los quase 100% prontos para analises, separando colunas com mais de uma informação e  
retirando caracteres especiais

In [91]:
df['Valor'] = df['Valor'].astype(str).apply(lambda x : x.split(' ')[1])
df['Iptu'] = df['IPTU_Condicao'].astype(str).apply(lambda x: x.split('+')[1])
df['Condominio'] = df['IPTU_Condicao'].astype(str).apply(lambda x: x.split('+')[0])
df['Endereco'] = df['Endereco'].astype(str).apply(lambda x : x.split(',')[1].split('·')[0])
df['Condominio'] = df['Condominio'].astype(str).apply(lambda x: x.split(' ')[1])
df['Caracteristicas'] = df['Caracteristicas'].astype(str).apply(lambda x:x.split(' · '))

In [93]:
for i,r in df.iterrows():
    if len(r['Caracteristicas']) == 2:
        df.loc[i,'m2']=r['Caracteristicas'][0].split(' ')[0]
        df.loc[i,'quartos']=r['Caracteristicas'][1].split(' ')[0]
    else:
        df.loc[i,'m2']=r['Caracteristicas'][0].split(' ')[0]
        df.loc[i,'quartos']=r['Caracteristicas'][1].split(' ')[0]
        df.loc[i,'vagas'] = r['Caracteristicas'][2].split(' ')[0]

In [94]:
df['vagas'].fillna(0,inplace=True)

In [95]:
df = df.drop(['IPTU_Condicao','Caracteristicas'], axis=1)

Nessa segunda parte deixar os dados 100% prontos vamos deixar as colunas nos tipos corretos para analise

In [ ]:
df.info()

In [ ]:
colunas_para_converter = ['Valor', 'Iptu', 'Condominio', 'm2', 'quartos', 'vagas']

for coluna in colunas_para_converter:
    df[coluna] = pd.to_numeric(df[coluna].str.replace('.', '').str.strip(), errors='coerce').fillna(0).astype(int)


In [ ]:
df.head()

In [99]:
df.to_csv('base.csv')